In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time

In [3]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

# Reshape and normalize images
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize to [-1, 1]

BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Batch and shuffle data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [6]:
# Define the generator model
def make_generator_model():
    model = tf.keras.Sequential()
    
    # Start with a dense layer
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Reshape into 7x7 feature maps
    model.add(layers.Reshape((7, 7, 256)))
    
    # Upsample to 14x14
    model.add(layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Upsample to 28x28
    model.add(layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Output layer
    model.add(layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh'))
    
    return model

generator = make_generator_model()

In [8]:
### Define the discriminator model
# The discriminator is a CNN that takes an image as input and outputs a single scalar
# indicating whether the image is real or fake.

def make_discriminator_model():
    model = tf.keras.Sequential()
    
    # Input layer
    model.add(layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=[28,28,1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    # Hidden layers
    model.add(layers.Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    # Output layer
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    
    return model

discriminator = make_discriminator_model()

In [9]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [10]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)
        
        # Generate and save images every epoch
        generate_and_save_images(generator, epoch + 1, seed)
        
        print(f'Epoch {epoch + 1}, Gen Loss: {gen_loss}, Disc Loss: {disc_loss}, Time: {time.time()-start}')

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    
    fig = plt.figure(figsize=(4,4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    
    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.close()

In [11]:
train(train_dataset, EPOCHS)

2025-05-23 14:09:42.467319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-05-23 14:10:03.316479: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Gen Loss: 0.7289643883705139, Disc Loss: 1.28121018409729, Time: 22.13076376914978


2025-05-23 14:10:19.696786: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, Gen Loss: 1.3145720958709717, Disc Loss: 0.8441545367240906, Time: 15.71422290802002


2025-05-23 14:10:35.435061: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2025-05-23 14:22:25.115104: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 48, Gen Loss: 0.9625252485275269, Disc Loss: 1.264991044998169, Time: 15.573347091674805


2025-05-23 14:22:40.746415: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 49, Gen Loss: 1.0464982986450195, Disc Loss: 1.3529059886932373, Time: 15.637904167175293
Epoch 50, Gen Loss: 0.946121096611023, Disc Loss: 1.2677092552185059, Time: 15.740614891052246


2025-05-23 14:22:56.491888: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [2]:
import glob
import imageio
import os
from PIL import Image
from IPython.display import display, Image as IPImage

def create_gif_from_existing_images():
    # Create directory if it doesn't exist
    os.makedirs('generated_images', exist_ok=True)
    
    # Get all epoch images from generated_images directory sorted by epoch number
    filenames = sorted(glob.glob('generated_images/image_at_epoch_*.png'), 
                   key=lambda x: int(x.split('_')[-1].split('.')[0]))
    
    if not filenames:
        print("No epoch images found in generated_images directory!")
        print("Please ensure your training script saves images to generated_images/")
        return
    
    anim_file = 'training_progress.gif'
    
    # Create the GIF writer
    with imageio.get_writer(anim_file, mode='I', duration=0.5) as writer:
        # Add each image to the GIF
        for filename in filenames:
            try:
                image = imageio.imread(filename)
                writer.append_data(image)
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue
        
        # Add the last frame a few more times to pause at the end
        if filenames:  # Check if there are any valid images
            for _ in range(5):
                try:
                    writer.append_data(imageio.imread(filenames[-1]))
                except Exception as e:
                    print(f"Error processing final frame: {str(e)}")
    
    # Verify the GIF was created
    if os.path.exists(anim_file):
        print(f"Successfully created GIF at {anim_file}")
        display(IPImage(filename=anim_file))
    else:
        print("Failed to create GIF")

# Execute this to create the GIF
create_gif_from_existing_images()

ModuleNotFoundError: No module named 'imageio'